In [39]:
# [1] imports
import torch
import torchvision
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
from tqdm import tqdm

In [40]:
# [2] Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# [3] Hyperparameters
NUM_CLASSES = 10
LEARNING_RATE = 1e-3
BATCH_SIZE = 1024
NUM_EPOCHS = 32


In [41]:
# [4] Get VGG16 Model
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x
    

model = torchvision.models.vgg16(weights="DEFAULT")
""" 
# Load pretrain model & modify it
#print(model)
# the dataset is different to the one the model is trained on
# Check dimension errors, after al conv layers we need to change avg pool
# and the classifier to accept 10 classes

model.avgpool = Identity()
#model.classifier = nn.Linear(512, 10)

# changing a particular classifier
# model.classifier[0] = nn.Linear(512, 10)
# for i in range (1, 7):
#     model.classifier[i] = Identity()

#model.classifier = nn.Sequential(nn.Linear(512,100),
#                                 nn.Dropout(p=0.5)
#                                 nn.Linear(100,10))

print(model)
model.to(DEVICE) 
"""

# I want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.
for param in model.parameters():
    param.requires_grad = False

model.avgpool = nn.Identity()
model.classifier = nn.Sequential(
    nn.Linear(512, 100), nn.ReLU(), nn.Linear(100, NUM_CLASSES)
)
model.to(DEVICE)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [42]:
# [5] Load data
train_dataset = datasets.CIFAR10(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified


In [43]:
# [7] Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [44]:
# [10] Checkpoint: SAVE and LOAD
def save_checkpoint(state, filename="checkpoint/VGG16_checkpoint.pth.tar"):
    print("saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint):
    print("loading checkpoint")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    # best current accuracy, ecc


LOAD_MODEL = True

if LOAD_MODEL == True:
    load_checkpoint(torch.load("checkpoint/VGG16_checkpoint.pth.tar"))



loading checkpoint


In [45]:
# [8] Train Network
for epoch in range(NUM_EPOCHS):
    losses = []

    # if accuracy is better tan an accuracy save model
    if epoch % 3 == 0:
        checkpoint = {'state_dict': model.state_dict(), 'optimizer' : optimizer.state_dict()}
        save_checkpoint(checkpoint)

    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=DEVICE)
        targets = targets.to(device=DEVICE)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}")

# Check accuracy on training & test to see how good our model

saving checkpoint


100%|██████████| 49/49 [00:11<00:00,  4.36it/s]


Cost at epoch 0 is 0.87850


100%|██████████| 49/49 [00:11<00:00,  4.42it/s]


Cost at epoch 1 is 0.87270


...

Cost at epoch 29 is 0.71769
saving checkpoint


100%|██████████| 49/49 [00:10<00:00,  4.74it/s]


Cost at epoch 30 is 0.71283


100%|██████████| 49/49 [00:10<00:00,  4.73it/s]

Cost at epoch 31 is 0.70850


In [46]:
# [9] Check Accuracy on training and test to see how good is my model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=DEVICE)
            y = y.to(device=DEVICE)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


check_accuracy(train_loader, model)

Checking accuracy on training data
Got 38234 / 50000 with accuracy 76.47
